# Visualize optical-IR stellar SEDs

Author: Melissa Graham

Date: Mon Oct 28 2024

RSP Image: Weekly 2024_37

Goal: Show how to display stellar SED with Rubin optical and Roman IR photometry.

## Introduction

### Warning!

The point of this notebook is just to show how to visualize the 
optical (Rubin) and infrared (Roman) SED of a few
simulated objects that are bright, and thus likely stars.

This notebook also does blackbody fits to the SEDs.
But it does so in a very unrigorous way!
**In particular, the affects of dust or other emission lines on the stellar
SED are not modelled or accounted for in the blackbody fits.**

### Basics

What's an SED? Spectral energy distribution: https://en.wikipedia.org/wiki/Spectral_energy_distribution

What's a blackbody? Thermal radiation of a continuous spectrum: https://en.wikipedia.org/wiki/Black-body_radiation

### The simulated data

The same simulation, DESC's Data Challenge 2 (DC2), is the basis for both
the simulated data products of Rubin's Data Preview 0, and the simulated
data for Roman Observatory presented in Troxel et al. (2023).

Thus, it is possible to cross-match the catalogs and obtain infrared
photometry for DP0 Objects.

Roman DC2 Simulated Images and Catalogs at IRSA IPAC:<br>
https://irsa.ipac.caltech.edu/data/theory/Roman/Troxel2023/overview.html

Troxel et al. (2023):<br>
https://academic.oup.com/mnras/article/522/2/2801/7076879?login=false

## Set up

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.coordinates import match_coordinates_sky
import astropy.units as u
from astropy.modeling import models, fitting
from lsst.rsp import get_tap_service, retrieve_query

In [ ]:
service = get_tap_service("tap")

## Retrieve data sets 

### Roman, Troxel et al. (2023)

In [ ]:
fnm = '/project/melissagraham2/troxel2023/dc2_det_52.21_-40.3.fits'
hdul = fits.open(fnm)
data = hdul[1].data

In [ ]:
roman_ra = np.asarray(data['alphawin_j2000'], dtype='float')
roman_dec = np.asarray(data['deltawin_j2000'], dtype='float')
roman_y = np.asarray(data['mag_auto_Y106'], dtype='float')
roman_j = np.asarray(data['mag_auto_J129'], dtype='float')
roman_h = np.asarray(data['mag_auto_H158'], dtype='float')
roman_f = np.asarray(data['mag_auto_F184'], dtype='float')
roman_ye = np.asarray(data['magerr_auto_Y106'], dtype='float')
roman_je = np.asarray(data['magerr_auto_J129'], dtype='float')
roman_he = np.asarray(data['magerr_auto_H158'], dtype='float')
roman_fe = np.asarray(data['magerr_auto_F184'], dtype='float')

In [ ]:
print('Number of Troxel objects: ', len(roman_ra))

In [ ]:
del fnm, hdul, data

### Rubin, Data Preview 0

In [ ]:
sra = str(np.round(np.mean(roman_ra), 3))
sde = str(np.round(np.mean(roman_dec), 3))
query = "SELECT coord_ra, coord_dec, "\
        "scisql_nanojanskyToAbMag(u_cModelFlux) AS umag, "\
        "scisql_nanojanskyToAbMag(g_cModelFlux) AS gmag, "\
        "scisql_nanojanskyToAbMag(r_cModelFlux) AS rmag, "\
        "scisql_nanojanskyToAbMag(i_cModelFlux) AS imag, "\
        "scisql_nanojanskyToAbMag(z_cModelFlux) AS zmag, "\
        "scisql_nanojanskyToAbMag(y_cModelFlux) AS ymag, "\
        "scisql_nanojanskyToAbMagSigma(u_cModelFlux, u_cModelFluxErr) AS umage, "\
        "scisql_nanojanskyToAbMagSigma(g_cModelFlux, g_cModelFluxErr) AS gmage, "\
        "scisql_nanojanskyToAbMagSigma(r_cModelFlux, r_cModelFluxErr) AS rmage, "\
        "scisql_nanojanskyToAbMagSigma(i_cModelFlux, i_cModelFluxErr) AS image, "\
        "scisql_nanojanskyToAbMagSigma(z_cModelFlux, z_cModelFluxErr) AS zmage, "\
        "scisql_nanojanskyToAbMagSigma(y_cModelFlux, y_cModelFluxErr) AS ymage "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), "\
        "CIRCLE('ICRS', "+sra+", "+sde+", 0.08)) = 1 "\
        "AND detect_isPrimary = 1"
print(query)
del sra, sde

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
results = job.fetch_result().to_table()
print('Number of DP0.2 objects: ', len(results))

In [ ]:
rubin_ra = np.asarray(results['coord_ra'], dtype='float')
rubin_dec = np.asarray(results['coord_dec'], dtype='float')
rubin_u = np.asarray(results['umag'], dtype='float')
rubin_g = np.asarray(results['gmag'], dtype='float')
rubin_r = np.asarray(results['rmag'], dtype='float')
rubin_i = np.asarray(results['imag'], dtype='float')
rubin_z = np.asarray(results['zmag'], dtype='float')
rubin_y = np.asarray(results['ymag'], dtype='float')
rubin_ue = np.asarray(results['umage'], dtype='float')
rubin_ge = np.asarray(results['gmage'], dtype='float')
rubin_re = np.asarray(results['rmage'], dtype='float')
rubin_ie = np.asarray(results['image'], dtype='float')
rubin_ze = np.asarray(results['zmage'], dtype='float')
rubin_ye = np.asarray(results['ymage'], dtype='float')

In [ ]:
del query, job, results

## Cross-match bright objects

Only use objects with y-band magnitude between 18 and 20. 

These are so bright they're likely to be stars.

In [ ]:
ru_x = np.where((rubin_y >= 18.0) & (rubin_y <= 20.0))[0]
ro_x = np.where((roman_y >= 18.0) & (roman_y <= 20.0))[0]

In [ ]:
rubin_coord = SkyCoord(ra=rubin_ra[ru_x]*u.degree, dec=rubin_dec[ru_x]*u.degree, frame='icrs')
roman_coord = SkyCoord(ra=roman_ra[ro_x]*u.degree, dec=roman_dec[ro_x]*u.degree, frame='icrs')

In [ ]:
idx, d2d, d3d = match_coordinates_sky(rubin_coord, roman_coord)

In [ ]:
max_off_arcsec = 0.5
rubin_rox = np.zeros(len(rubin_y), dtype='int') - 1
for i in range(len(ru_x)):
    if d2d.arcsec[i] < 0.5:
        rubin_rox[ru_x[i]] = ro_x[idx[i]]

In [ ]:
tx = np.where(rubin_rox >= 0)[0]
print('Number of bright Rubin objects with a Roman object within 0.5": ', len(tx))
del tx

### Count number of detection filters

Identify y-bright objects that are detected with magnitudes < 25 in all other filters.

In [ ]:
rubin_nfilt = np.zeros(len(rubin_ra), dtype='int')
tx = np.where(rubin_rox[ru_x] >= 0)[0]
for x in tx:
    count = 0
    if rubin_u[ru_x[x]] < 25:
        count += 1
    if rubin_g[ru_x[x]] < 25:
        count += 1
    if rubin_r[ru_x[x]] < 25:
        count += 1
    if rubin_i[ru_x[x]] < 25:
        count += 1
    if rubin_z[ru_x[x]] < 25:
        count += 1
    if roman_j[rubin_rox[ru_x[x]]] < 25:
        count += 1
    if roman_h[rubin_rox[ru_x[x]]] < 25:
        count += 1
    if roman_f[rubin_rox[ru_x[x]]] < 25:
        count += 1
    rubin_nfilt[ru_x[x]] = count
del tx

In [ ]:
tx = np.where(rubin_nfilt == 8)[0]
print('Number of y-bright objects detected in 8 filters: ', len(tx))
del tx

## Blackbody fit for one (potential) star

> **WARNING** These blackbody fits do not include the effects of
> dust or emission lines on the SED.

Pick one of the potential stars with detections in all filters to use.

In [ ]:
tx = np.where(rubin_nfilt == 8)[0]
my_j = 14
my_i = tx[my_j]
del tx

### Define the filters' effective wavelengths

https://github.com/lsst/throughputs/blob/main/examples/LSST%20Throughputs%20Curves.ipynb

LSST effective wavelength per filter in nanometers.

In [ ]:
lsst_eff_wl = {'u': 370.9,'g': 476.7,'r': 619.4,
               'i': 753.9,'z': 866.8,'y': 973.9}

https://roman-docs.stsci.edu/roman-instruments-home/wfi-imaging-mode-user-guide/wfi-design/wfi-optical-elements

Roman pivot wavelength per filter in microns.

In [ ]:
roman_eff_wl = {'Y106': 1.0567, 'J129': 1.2901,
                'H158': 1.5749, 'F184': 1.8394}

### Fit LSST filters only

First just plot the photometry.

In [ ]:
fig = plt.figure(figsize=(6, 4))
plt.plot(lsst_eff_wl['u'], rubin_u[my_i], 'o')
plt.plot(lsst_eff_wl['g'], rubin_g[my_i], 'o')
plt.plot(lsst_eff_wl['r'], rubin_r[my_i], 'o')
plt.plot(lsst_eff_wl['i'], rubin_i[my_i], 'o')
plt.plot(lsst_eff_wl['z'], rubin_z[my_i], 'o')
plt.plot(lsst_eff_wl['y'], rubin_y[my_i], 'o')
plt.plot(roman_eff_wl['Y106']*1000.0, roman_y[rubin_rox[my_i]], 's')
plt.plot(roman_eff_wl['J129']*1000.0, roman_j[rubin_rox[my_i]], 's')
plt.plot(roman_eff_wl['H158']*1000.0, roman_h[rubin_rox[my_i]], 's')
plt.plot(roman_eff_wl['F184']*1000.0, roman_f[rubin_rox[my_i]], 's')
plt.xlabel('Filter effective wavelength [nm]')
plt.ylabel('Magnitude')
plt.gca().invert_yaxis()
plt.show()

Use `astropy` code to fit a blackbody to the optical fluxes.

First create input arrays.

In [ ]:
lsst_wls = np.asarray([lsst_eff_wl['u']*10.0, lsst_eff_wl['g']*10.0,
                       lsst_eff_wl['r']*10.0, lsst_eff_wl['i']*10.0,
                       lsst_eff_wl['z']*10.0, lsst_eff_wl['y']*10.0]) * u.Angstrom
lsst_flx = np.asarray([1e-9 * np.power(10, (rubin_u[my_i] - 31.4)/(-2.5)),
                       1e-9 * np.power(10, (rubin_g[my_i] - 31.4)/(-2.5)),
                       1e-9 * np.power(10, (rubin_r[my_i] - 31.4)/(-2.5)),
                       1e-9 * np.power(10, (rubin_i[my_i] - 31.4)/(-2.5)),
                       1e-9 * np.power(10, (rubin_z[my_i] - 31.4)/(-2.5)),
                       1e-9 * np.power(10, (rubin_y[my_i] - 31.4)/(-2.5))]) * u.Jy

In [ ]:
blackbody_model = models.BlackBody(temperature=5000 * u.K)
fitter = fitting.LevMarLSQFitter()
fit_result = fitter(blackbody_model, lsst_wls, lsst_flx)
fit_result

In [ ]:
fig = plt.figure(figsize=(6, 4))
plt.plot(lsst_wls, lsst_flx, 'o', label='Data')
plt.plot(lsst_wls, fit_result(lsst_wls), label='Fit')
plt.xlabel('Wavelength (Angstrom)')
plt.ylabel('Flux (Jy)')
plt.legend()
plt.show()

In [ ]:
del lsst_wls, lsst_flx, blackbody_model, fitter, fit_result

### Fit LSST and Roman filters

Use `astropy` code to fit a blackbody to the optical + IR fluxes. Note the temperature is different from the optical-only fit.

In [ ]:
all_wls = np.asarray([lsst_eff_wl['u']*10.0, lsst_eff_wl['g']*10.0,
                      lsst_eff_wl['r']*10.0, lsst_eff_wl['i']*10.0,
                      lsst_eff_wl['z']*10.0, lsst_eff_wl['y']*10.0,
                      roman_eff_wl['Y106']*10000.0, roman_eff_wl['J129']*10000.0,
                      roman_eff_wl['H158']*10000.0, roman_eff_wl['F184']*10000.0]) * u.Angstrom
all_flx = np.asarray([1e-9 * np.power(10, (rubin_u[my_i] - 31.4)/(-2.5)),
                      1e-9 * np.power(10, (rubin_g[my_i] - 31.4)/(-2.5)),
                      1e-9 * np.power(10, (rubin_r[my_i] - 31.4)/(-2.5)),
                      1e-9 * np.power(10, (rubin_i[my_i] - 31.4)/(-2.5)),
                      1e-9 * np.power(10, (rubin_z[my_i] - 31.4)/(-2.5)),
                      1e-9 * np.power(10, (rubin_y[my_i] - 31.4)/(-2.5)),
                      1e-9 * np.power(10, (roman_y[rubin_rox[my_i]] - 31.4)/(-2.5)),
                      1e-9 * np.power(10, (roman_h[rubin_rox[my_i]] - 31.4)/(-2.5)),
                      1e-9 * np.power(10, (roman_h[rubin_rox[my_i]] - 31.4)/(-2.5)),
                      1e-9 * np.power(10, (roman_f[rubin_rox[my_i]] - 31.4)/(-2.5))]) * u.Jy

In [ ]:
blackbody_model = models.BlackBody(temperature=5000 * u.K)
fitter = fitting.LevMarLSQFitter()
fit_result = fitter(blackbody_model, all_wls, all_flx)
fit_result

In [ ]:
fig = plt.figure(figsize=(6, 4))
plt.plot(all_wls, all_flx, 'o', label='Data')
plt.plot(all_wls, fit_result(all_wls), label='Fit')
plt.xlabel('Wavelength (Angstrom)')
plt.ylabel('Flux (Jy)')
plt.legend()
plt.show()

In [ ]:
del all_wls, all_flx, blackbody_model, fitter, fit_result

## Blackbody fits for all (potential) stars

In [ ]:
all_wls = np.asarray([lsst_eff_wl['u']*10.0, lsst_eff_wl['g']*10.0,
                      lsst_eff_wl['r']*10.0, lsst_eff_wl['i']*10.0,
                      lsst_eff_wl['z']*10.0, lsst_eff_wl['y']*10.0,
                      roman_eff_wl['Y106']*10000.0, roman_eff_wl['J129']*10000.0,
                      roman_eff_wl['H158']*10000.0, roman_eff_wl['F184']*10000.0]) * u.Angstrom
lssti = np.asarray([0,1,2,3,4,5], dtype='int')
blackbody_model = models.BlackBody(temperature=5000 * u.K)
fitter = fitting.LevMarLSQFitter()

In [ ]:
temp1 = []
temp2 = []
tx = np.where(rubin_nfilt == 8)[0]
for x in tx:
    all_flx = np.asarray([1e-9 * np.power(10, (rubin_u[x] - 31.4)/(-2.5)),
                          1e-9 * np.power(10, (rubin_g[x] - 31.4)/(-2.5)),
                          1e-9 * np.power(10, (rubin_r[x] - 31.4)/(-2.5)),
                          1e-9 * np.power(10, (rubin_i[x] - 31.4)/(-2.5)),
                          1e-9 * np.power(10, (rubin_z[x] - 31.4)/(-2.5)),
                          1e-9 * np.power(10, (rubin_y[x] - 31.4)/(-2.5)),
                          1e-9 * np.power(10, (roman_y[rubin_rox[x]] - 31.4)/(-2.5)),
                          1e-9 * np.power(10, (roman_h[rubin_rox[x]] - 31.4)/(-2.5)),
                          1e-9 * np.power(10, (roman_h[rubin_rox[x]] - 31.4)/(-2.5)),
                          1e-9 * np.power(10, (roman_f[rubin_rox[x]] - 31.4)/(-2.5))]) * u.Jy
    fit_result1 = fitter(blackbody_model, all_wls[lssti], all_flx[lssti])
    temp1.append(fit_result1.temperature.value)
    fit_result2 = fitter(blackbody_model, all_wls, all_flx)
    temp2.append(fit_result2.temperature.value)
    del all_flx, fit_result1, fit_result2
del tx

In [ ]:
temps_opt = np.asarray(temp1, dtype='float')
temps_optIR = np.asarray(temp2, dtype='float')
del temp1, temp2

In [ ]:
fig = plt.figure(figsize=(6, 4))
plt.plot([3000, 6500], [3000, 6500], lw=1, ls='solid', color='lightgrey')
plt.plot(temps_opt, temps_optIR, 'o', ms=5, mew=0, alpha=0.8, color='grey')
plt.plot(temps_opt[my_j], temps_optIR[my_j], '*', ms=15, mew=0, alpha=1,
         color='black', label='the first object fit')
plt.xlabel('Optical only fit temperature')
plt.ylabel('Optical+IR fit temperature')
plt.show()

Above, the black star marks the object for which the SED fits were shown in the section above.

### Suggested exercise

Go back and use a `my_j` of 0 or 1 to see lower-temperature SED fits.